In [74]:
import pandas as pd
import os

data_folder = 'OfficerIDbyYear'
csv_files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Load CSV (Employee data) into a dictionary of DataFrames (one for each year)
employee_data = {}
years = range(2010, 2022)

for csv_file in csv_files:
    # Check if the filename matches the pattern
    if 'Order of Councilor Arroyo' in csv_file:  
        # Extract the year from the filename by splitting based on space and dot
        year = csv_file.split(' - ')[-1].split('.')[2] 
        
        # Load the data for that year
        file_path = os.path.join(data_folder, csv_file)
        year_data = pd.read_csv(file_path)
        
        # remove officer if they are inactive
        year_data.columns = year_data.columns.str.strip().str.lower()
            
        employee_data[year] = year_data[year_data["reason inactive"].isnull()]

# print(employee_data['2012'].head())
print(employee_data.keys())

dict_keys(['2021', '2020', '2017', '2011', '2010', '2013', '2018', '2016', '2022', '2012', '2015', '2019', '2014'])


In [75]:
import string

# load incident data
incident_data = pd.read_csv("normalized_cases.csv")

# create year column 
incident_data['Incident Year'] = pd.to_datetime(incident_data['Inc: Received date']).dt.year
incident_data['Incident Year'] = incident_data['Incident Year'].dropna()
incident_data['Incident Year'] = incident_data['Incident Year'].astype(int)

# create a full name column
incident_data['Middle_Initial'] = incident_data['Middle_Initial'].astype(str)
incident_data['Middle_Initial'] = incident_data['Middle_Initial'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
incident_data['Full Name'] = incident_data['First_Name'] + " " + incident_data['Middle_Initial'] + " " + incident_data['Off: Last name']

incident_data.head()

,Unnamed: 0,Inc: IA No,Inc: Incident type,Inc: Received date,Inc: Occurred date,OffSnp: Title/rank,Off: First name,Off: Last name,Alg: Allegation,Alg: Finding,Act: Action taken,Act: Days/hours suspended,Act: Action taken date,First_Name,Middle_Initial,Incident Year,Full Name
0,0,IAD2011-0003,Citizen complaint,01/05/2011,06/18/2009,Sergeant Detective,Albert M,Terestre,Use of Force,Exonerated,NaN,0.0,NaN,Albert,M,2011,Albert M Terestre
1,1,IAD2011-0003,Citizen complaint,01/05/2011,06/18/2009,Police Officer,John K,Rogers,Use of Force,Exonerated,NaN,0.0,NaN,John,K,2011,John K Rogers
2,2,IAD2011-0004,Citizen complaint,01/05/2011,01/01/2011,Detective,Scott M,Mackie,Conformance to Laws,Unfounded,NaN,NaN,NaN,Scott,M,2011,Scott M Mackie
3,3,IAD2011-0011,Citizen complaint,01/07/2011,01/03/2011,Police Officer,Michael J.,Olszak,Neg.Duty/Unreasonable Judge,Exonerated,NaN,0.0,NaN,Michael,J,2011,Michael J Olszak
4,4,IAD2011-0011,Citizen complaint,01/07/2011,01/03/2011,Police Officer,Angel,Figueroa,Neg.Duty/Unreasonable Judge,Exonerated,NaN,0.0,NaN,Angel,nan,2011,Angel nan Figueroa


In [76]:
def get_employee_id(row):
    # Get the year from incident data
    year = str(row['Incident Year'])
    
    # Check if the year is present in the employee data for that year
    year_data = employee_data.get(year)
    if year_data is not None:
        # Create Full Name in Employee Data
        year_data['Full Name'] = year_data['first name'] + " " + year_data['middle'] + " " + year_data['last']
        
        # Find matching Full Name and return Employee ID
        matching_row = year_data[year_data['Full Name'] == row['Full Name']]
        
        if not matching_row.empty:
            return matching_row.iloc[0]['id']
        
    return None  

In [79]:
# Apply the function to the Incident Data to add the Employee ID
incident_data['Employee ID'] = incident_data.apply(get_employee_id, axis=1)
incident_data = incident_data.dropna(subset=['Employee ID'])
incident_data['Employee ID'] = incident_data['Employee ID'].astype('Int64')

# Save the result to a new CSV file
incident_data.to_csv("incident_with_employee_ids.csv", index=False)
print(incident_data.head())

    Unnamed: 0    Inc: IA No Inc: Incident type Inc: Received date  \
0            0  IAD2011-0003  Citizen complaint         01/05/2011   
1            1  IAD2011-0003  Citizen complaint         01/05/2011   
8            8  IAD2011-0013  Citizen complaint         01/07/2011   
16          16  IAD2011-0023  Citizen complaint         01/14/2011   
21          21  IAD2011-0039  Citizen complaint         01/22/2011   

   Inc: Occurred date  OffSnp: Title/rank Off: First name Off: Last name  \
0          06/18/2009  Sergeant Detective        Albert M       Terestre   
1          06/18/2009      Police Officer          John K         Rogers   
8          10/27/2010      Police Officer        Andrew B          Blake   
16         11/01/2012           Detective         Laura A        Delaney   
21         01/22/2011      Police Officer       Richard L         Medina   

                Alg: Allegation   Alg: Finding Act: Action taken  \
0                  Use of Force     Exonerated        